In [1]:
import sys
sys.path.append('../')
from database.fetch_data import *
import pandas as pd
from itertools import product
import string
from probabilistic_database_creation.rv_probability_assignments import *

In [2]:
#These constants can be changed dynamically

#Name of the inconsistent DB
table_name = "car_owner_corrupted"

#determinant attribute (LHS of FD): In person -> {model,color}, determinant = person
#TODO: Extend for multiple determinants
determinant_attributes = ["person"]

#dependent attributes (RHS of FD): In person -> {model,color}, dependent = model,color
dependent_attributes = ["model","color"]

In [3]:
all_records = fetch_data_results(fetch_all_records(table_name,determinant_attributes,dependent_attributes))
data = pd.DataFrame(all_records, columns=determinant_attributes+dependent_attributes)

In [4]:
dependent_attributes_mappings = generate_rvs(data,dependent_attributes)
rv_assignments = create_rv_assignments(data,dependent_attributes_mappings,determinant_attributes)

In [9]:
rv_assignments_CWA = create_rv_assignments_with_prob_CWA(data,dependent_attributes_mappings,determinant_attributes,dependent_attributes)

for determinant_attribute, assignments in rv_assignments_CWA.items():
    print(f"{determinant_attribute}:")
    for rv, prob in assignments.items():
        print(f"  {rv}: {prob}")

Frank:
  a1=1: {'value': 'Toyota', 'prob': 1.0}
  a1=2: {'value': 'Honda', 'prob': 0}
  a1=3: {'value': 'Mazda', 'prob': 0}
  b1=1: {'value': 'Red', 'prob': 0.5}
  b1=2: {'value': 'Blue', 'prob': 0.5}
  b1=3: {'value': 'Green', 'prob': 0}
Billy:
  a2=1: {'value': 'Toyota', 'prob': 0}
  a2=2: {'value': 'Honda', 'prob': 1.0}
  a2=3: {'value': 'Mazda', 'prob': 0}
  b2=1: {'value': 'Red', 'prob': 0}
  b2=2: {'value': 'Blue', 'prob': 1.0}
  b2=3: {'value': 'Green', 'prob': 0}
Jimmy:
  a3=1: {'value': 'Toyota', 'prob': 0}
  a3=2: {'value': 'Honda', 'prob': 0}
  a3=3: {'value': 'Mazda', 'prob': 1.0}
  b3=1: {'value': 'Red', 'prob': 0}
  b3=2: {'value': 'Blue', 'prob': 0}
  b3=3: {'value': 'Green', 'prob': 1.0}
Thomas:
  a4=1: {'value': 'Toyota', 'prob': 1.0}
  a4=2: {'value': 'Honda', 'prob': 0}
  a4=3: {'value': 'Mazda', 'prob': 0}
  b4=1: {'value': 'Red', 'prob': 0.5}
  b4=2: {'value': 'Blue', 'prob': 0.5}
  b4=3: {'value': 'Green', 'prob': 0}
Betty:
  a5=1: {'value': 'Toyota', 'prob': 0.33

In [10]:
rv_assignments_SWA = create_rv_assignments_with_prob_SWA(data,dependent_attributes_mappings,determinant_attributes,dependent_attributes)
for determinant_attribute, assignments in rv_assignments_SWA.items():
    print(f"{determinant_attribute}:")
    for rv, prob in assignments.items():
        print(f"  {rv}: {prob}")

Frank:
  a1=1: {'value': 'Toyota', 'prob': 0.8}
  a1=2: {'value': 'Honda', 'prob': 0.1}
  a1=3: {'value': 'Mazda', 'prob': 0.1}
  b1=1: {'value': 'Red', 'prob': 0.4}
  b1=2: {'value': 'Blue', 'prob': 0.4}
  b1=3: {'value': 'Green', 'prob': 0.2}
Billy:
  a2=1: {'value': 'Toyota', 'prob': 0}
  a2=2: {'value': 'Honda', 'prob': 1}
  a2=3: {'value': 'Mazda', 'prob': 0}
  b2=1: {'value': 'Red', 'prob': 0}
  b2=2: {'value': 'Blue', 'prob': 1}
  b2=3: {'value': 'Green', 'prob': 0}
Jimmy:
  a3=1: {'value': 'Toyota', 'prob': 0}
  a3=2: {'value': 'Honda', 'prob': 0}
  a3=3: {'value': 'Mazda', 'prob': 1}
  b3=1: {'value': 'Red', 'prob': 0}
  b3=2: {'value': 'Blue', 'prob': 0}
  b3=3: {'value': 'Green', 'prob': 1}
Thomas:
  a4=1: {'value': 'Toyota', 'prob': 0.8}
  a4=2: {'value': 'Honda', 'prob': 0.1}
  a4=3: {'value': 'Mazda', 'prob': 0.1}
  b4=1: {'value': 'Red', 'prob': 0.4}
  b4=2: {'value': 'Blue', 'prob': 0.4}
  b4=3: {'value': 'Green', 'prob': 0.2}
Betty:
  a5=1: {'value': 'Toyota', 'prob': 